<a href="https://colab.research.google.com/github/edwardtatem38-pixel/CNN-Image-Classification-/blob/main/nestle_hr_assistant_changed_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# core rag libraries and interface
!pip install -q protobuf==5.29.1 # Pin protobuf to a compatible version that resolves conflicts
!pip install -q langchain pypdf chromadb gradio --force-reinstall # Reinstall to pick up correct protobuf

# local embeddings model
!pip install -q sentence-transformers

# local llm runner (crucial for cpu inference)
# '--force-reinstall' flag to ensure compatibility in colab
!pip install -q llama-cpp-python --force-reinstall # Reinstall to pick up correct protobuf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.8/102.8 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.2/328.2 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 120.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.0/23.0 MB 79.4 

In [ ]:
# ----------------------------------------------------------------------
# --- DOWNLOAD THE LOCAL LLM (GGUF Format) ---
# We will use the highly capable and relatively small Mistral 7B Instruct model.
# Q4_K_M is a good quantization balance of speed and accuracy for CPU use.
# This file is ~4.3 GB and will be downloaded directly to the Colab environment.

MODEL_REPO_ID = "TheBloke/Mistral-7B-Instruct-v0.2-GGUF"
MODEL_FILENAME = "mistral-7b-instruct-v0.2.Q4_K_M.gguf"
CHROMA_DB_DIR = "./chroma_db_local"
print(f"Downloading {MODEL_FILENAME}...")
PDF_FILE = "the_nestle_hr_policy_pdf_2012 (1).pdf"

print("Download complete.")
# ----------------------------------------------------------------------

Download complete.


In [ ]:
# Install the dedicated package for text splitters
!pip install -q langchain-text-splitters

In [ ]:
# core rag libraries and interface
!pip install -q protobuf==5.29.1
# Added langchain-community and langchain-huggingface explicitly
!pip install -q langchain langchain-community langchain-text-splitters pypdf chromadb gradio --force-reinstall

# local embeddings model
!pip install -q sentence-transformers langchain-huggingface

# local llm runner
!pip install -q llama-cpp-python --force-reinstall

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.1/75.1 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 49.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 86.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.9/51.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 100.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.4/242.4 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 609.9/609.9 kB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.1/256.1 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.6/221.6 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.3/377.3 kB 32.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currentl

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
# *** THIS IS THE CRITICAL CORRECTION ***
from langchain_text_splitters import RecursiveCharacterTextSplitter
# **************************************
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

print("Starting document processing and vectorization...")

PDF_FILE = "/content/the_nestle_hr_policy_pdf_2012 (1).pdf"

loader = PyPDFLoader(PDF_FILE)
documents = loader.load()

# 2. Split the document
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)
print(f"Document split into {len(chunks)} chunks.")

# 3. Load the Local Embeddings Model (CPU-friendly)
model_name = "sentence-transformers/all-MiniLM-L6-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}

embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

# 4. Create the Vector Store. This is the search index for your document.
vectorstore = Chroma.from_documents(chunks, embeddings, persist_directory="./chroma_db_local")
print("Vector store creation complete.")

/usr/local/lib/python3.12/dist-packages/jax/_src/cloud_tpu_init.py:86: UserWarning: Transparent hugepages are not enabled. TPU runtime startup and shutdown time should be significantly improved on TPU v5e and newer. If not already set, you may need to enable transparent hugepages in your VM image (sudo sh -c "echo always > /sys/kernel/mm/transparent_hugepage/enabled")
  warnings.warn(


Starting document processing and vectorization...
Document split into 20 chunks.


/tmp/ipython-input-765880632.py:25: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Vector store creation complete.


### Upload the PDF File

Please use the following code cell to upload the PDF file `the_nestle_hr_policy_pdf_2012 (1).pdf`. Once uploaded, the previous error related to the file not being found should be resolved.

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print(f'User uploaded file "{fn}" with length {len(uploaded[fn])} bytes')


Saving the_nestle_hr_policy_pdf_2012 (1).pdf to the_nestle_hr_policy_pdf_2012 (1).pdf
User uploaded file "the_nestle_hr_policy_pdf_2012 (1).pdf" with length 1566930 bytes


In [ ]:
  from langchain_community.llms import LlamaCpp
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

print("Setting up the pure CPU LLM and RAG chain using LCEL...")

Setting up the pure CPU LLM and RAG chain using LCEL...


In [ ]:
# Filename for the downloaded GGUF model (Assuming this variable is still available)
MODEL_FILENAME = "mistral-7b-instruct-v0.2.Q4_K_M.gguf"

In [ ]:
# -----------------------------------------------------------
# CRITICAL NEW LOGIC: Download LLM from Hugging Face Hub
# This bypasses the 1 GB repository limit by downloading at runtime.
# -----------------------------------------------------------

MODEL_REPO_ID = "TheBloke/Mistral-7B-Instruct-v0.2-GGUF" # Added definition for MODEL_REPO_ID

# 1. DOWNLOAD THE MODEL FILE TO THE PERSISTENT DISK
print(f"Downloading LLM from {MODEL_REPO_ID}...")
from huggingface_hub import hf_hub_download
model_path = hf_hub_download(
    repo_id=MODEL_REPO_ID,
    filename=MODEL_FILENAME,
    local_dir_use_symlinks=False # Important for the Space environment
)

# 2. Load the LlamaCpp Model (CPU only)
print(f"Loading LlamaCpp Model from downloaded path: {model_path}...")
llm = LlamaCpp(
    model_path=model_path,  # <-- CRITICAL: Uses the path returned by the download
    n_gpu_layers=0,
    n_ctx=4096,
    n_threads=2,
    temperature=0.1,
    verbose=False,
)
# -----------------------------------------------------------


Loading LlamaCpp Model from downloaded path: /root/.cache/huggingface/hub/models--TheBloke--Mistral-7B-Instruct-v0.2-GGUF/snapshots/3a6fbf4a41a1d52e415a4958cde6856d34b2db93/mistral-7b-instruct-v0.2.Q4_K_M.gguf...


llama_context: n_batch is less than GGML_KQ_MASK_PAD - increasing to 64
llama_context: n_ctx_per_seq (4096) < n_ctx_train (32768) -- the full capacity of the model will not be utilized


In [ ]:
# 2. Define the Prompt Template
template = """Use the following pieces of context to answer the user's question.
If you cannot find the answer in the context, clearly state: "I cannot find the relevant information in the Nestlé HR Policy document." Do not try to make up an answer.

{context}

Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

In [ ]:
# 3. Create the Retriever (Assuming 'vectorstore' was successfully created in Step 3)
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})

In [ ]:
# 4. Define the RAG chain using LCEL
def format_docs(docs):
    """Formats the retrieved documents into a single string for the prompt context."""
    return "\n\n".join(doc.page_content for doc in docs)

qa_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | QA_CHAIN_PROMPT
    | llm
    | StrOutputParser()
)

print("RAG Question-Answering chain (LCEL) is ready.")

RAG Question-Answering chain (LCEL) is ready.


In [ ]:
import gradio as gr

In [ ]:
# This function wraps the RAG chain for Gradio
def respond(message, history):
    # This call triggers the RAG process: retrieve context, then generate answer
    result = qa_chain.invoke(message) # LCEL chain takes the question directly
    answer = result
    return answer

In [ ]:
# Create the Chat Interface
iface = gr.ChatInterface(
    fn=respond,
    title="Nestlé HR Policy Chatbot (Pure CPU)",
    description="Ask questions about the uploaded Nestlé HR Policy document. Uses local LLM (Mistral 7B GGUF) on CPU only.",
    examples=["What is the sick leave policy?", "Who is eligible for the retirement benefits?"],
)

# Launch the interface. Colab will provide a public URL for interaction.
print("Launching Gradio Interface...")
iface.launch(share=True)

Launching Gradio Interface...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b98a800e16207beb7d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
